In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# 读取输入数据
input_file = "input.csv"  # 输入 CSV 文件
output_file = "output.csv"  # 输出 CSV 文件

df_input = pd.read_csv(input_file)
df_output = pd.read_csv(output_file)

# 选择输入特征
input_features = ["Time", "AntriebsmomentVA", "AntriebsmomentHA", "Lenkwinkel", "BremsmomentVA", "BremsmomentHA", "Geschw_S"]
output_features = ["Time", "KS_X", "KS_Y", "Gierwinkel", "Geschw_X", "Geschw_Y", "Giergeschwindigkeit"]

# 归一化数据
scalers = {}
for feature in input_features:
    scaler = MinMaxScaler()
    df_input[feature] = scaler.fit_transform(df_input[[feature]])
    scalers[feature] = scaler

for feature in output_features:
    scaler = MinMaxScaler()
    df_output[feature] = scaler.fit_transform(df_output[[feature]])
    scalers[feature] = scaler

# 创建时间序列数据集
def create_sequences(X, y, time_steps=10):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i + time_steps])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

# 第一阶段：预测 Giergeschwindigkeit
X1, y1 = create_sequences(df_input[input_features].values, df_output["Giergeschwindigkeit"].values)

model1 = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(X1.shape[1], X1.shape[2])),
    LSTM(50, activation='relu'),
    Dense(1)
])

model1.compile(optimizer='adam', loss='mse')
model1.fit(X1, y1, epochs=10, batch_size=32, verbose=1)

giergeschwindigkeit_pred = model1.predict(X1)

# 第二阶段：预测 Gierwinkel
X2 = np.hstack([df_input[["Time","Lenkwinkel"]].values[len(df_input) - len(giergeschwindigkeit_pred):], giergeschwindigkeit_pred])
X2 = X2.reshape(X2.shape[0], 1, X2.shape[1])
y2 = df_output["Gierwinkel"].values[len(df_output) - len(X2):]

model2 = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(X2.shape[1], X2.shape[2])),
    LSTM(50, activation='relu'),
    Dense(1)
])

model2.compile(optimizer='adam', loss='mse')
model2.fit(X2, y2, epochs=10, batch_size=32, verbose=1)

gierwinkel_pred = model2.predict(X2)

# 第三阶段：预测 Geschw_X, Geschw_Y
X3 = np.hstack([df_input[["Time", "Lenkwinkel", "Geschw_S"]].values[len(df_input) - len(gierwinkel_pred):], gierwinkel_pred])
X3 = X3.reshape(X3.shape[0], 1, X3.shape[1])
y3 = df_output[["Geschw_X", "Geschw_Y"]].values[len(df_output) - len(X3):]

model3 = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(X3.shape[1], X3.shape[2])),
    LSTM(50, activation='relu'),
    Dense(2)
])

model3.compile(optimizer='adam', loss='mse')
model3.fit(X3, y3, epochs=10, batch_size=32, verbose=1)

geschw_pred = model3.predict(X3)

# 第四阶段：预测 KS_X, KS_Y
X4 = np.hstack([df_input[["Time", "Lenkwinkel"]].values[len(df_input) - len(geschw_pred):], geschw_pred])
X4 = X4.reshape(X4.shape[0], 1, X4.shape[1])
y4 = df_output[["KS_X", "KS_Y"]].values[len(df_output) - len(X4):]

model4 = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(X4.shape[1], X4.shape[2])),
    LSTM(50, activation='relu'),
    Dense(2)
])

model4.compile(optimizer='adam', loss='mse')
model4.fit(X4, y4, epochs=10, batch_size=32, verbose=1)

ks_pred = model4.predict(X4)

# 反归一化
giergeschwindigkeit_pred = scalers["Giergeschwindigkeit"].inverse_transform(giergeschwindigkeit_pred)
gierwinkel_pred = scalers["Gierwinkel"].inverse_transform(gierwinkel_pred)
geschw_pred = scalers["Geschw_X"].inverse_transform(geschw_pred)
ks_pred = scalers["KS_X"].inverse_transform(ks_pred)

# 修正 Time 使其与输入保持一致
output_df = pd.DataFrame({
    "KS_X": ks_pred[:, 0],
    "KS_Y": ks_pred[:, 1],
    "Giergeschwindigkeit": giergeschwindigkeit_pred[:, 0],
    "Gierwinkel": gierwinkel_pred[:, 0],
    "Geschw_X": geschw_pred[:, 0],
    "Geschw_Y": geschw_pred[:, 1]
})

output_df.to_csv("predictions.csv", index=False)

# 只保留最后 25000 行
if len(output_df) > 25000:
    output_df = output_df.iloc[-25000:]

output_df.to_csv("predictions_trimmed.csv", index=False)

print("预测完成，结果已保存到 predictions.csv！")
print("已生成 predictions_trimmed.csv，仅包含最后 25000 行数据！")


Epoch 1/10


C:\Users\Zelin\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3907/3907 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - loss: 0.0048
Epoch 2/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - loss: 4.6451e-05
Epoch 3/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - loss: 4.3642e-05
Epoch 4/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - loss: 3.4169e-05
Epoch 5/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - loss: 3.9354e-05
Epoch 6/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - loss: 3.2968e-05
Epoch 7/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - loss: 1.8801e-05
Epoch 8/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - loss: 2.0584e-05
Epoch 9/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - loss: 1.9105e-05
Epoch 10/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - loss: 2.0142e-05
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step
Epoch 1/10


C:\Users\Zelin\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3907/3907 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - loss: 0.0155
Epoch 2/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 1.3254e-05
Epoch 3/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 8.9311e-06
Epoch 4/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 8.5880e-06
Epoch 5/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 7.2775e-06
Epoch 6/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 6.2347e-06
Epoch 7/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 6.0355e-06
Epoch 8/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 5.0576e-06
Epoch 9/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 4.3485e-06
Epoch 10/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 4.7334e-06
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 3s 746us/step
Epoch 1/10


C:\Users\Zelin\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3907/3907 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - loss: 0.0117
Epoch 2/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0016
Epoch 3/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0012
Epoch 4/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 9.7301e-04
Epoch 5/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 8.9673e-04
Epoch 6/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 8.5980e-04
Epoch 7/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 8.3616e-04
Epoch 8/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 8.1387e-04
Epoch 9/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 7.9801e-04
Epoch 10/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 7.8070e-04
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 3s 722us/step
Epoch 1/10


C:\Users\Zelin\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3907/3907 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - loss: 0.0221
Epoch 2/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 7.6730e-04
Epoch 3/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 4.0233e-04
Epoch 4/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 1.9582e-04
Epoch 5/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 1.7089e-04
Epoch 6/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 1.5798e-04
Epoch 7/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 1.4627e-04
Epoch 8/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 1.3934e-04
Epoch 9/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 1.3007e-04
Epoch 10/10
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 1.2505e-04
3907/3907 ━━━━━━━━━━━━━━━━━━━━ 3s 752us/step
预测完成，结果已保存到 predictions.csv！
已生成 predictions_trimmed.csv，仅包含最后 25000 行数据！
